<a href="https://colab.research.google.com/github/samyuktsriram/ClutchMomentsFootballAnalysis/blob/main/DistilGPT2_for_Match_Summary_Samyukt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

In [6]:
import json
import os
import pandas as pd

from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset

from transformers import TFAutoModelForCausalLM, TrainingArguments, Trainer
from transformers import create_optimizer, AdamWeightDecay

import tensorflow as tf


In [7]:
corpus_path = "/content/combined_raw.json"

with open(corpus_path) as f:
  
  a = json.load(f)

  #sequence = ''

  #for row in a:
    #sequence += row['Report']

In [27]:
#Rough look at data
df = pd.DataFrame.from_dict(a)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      1598 non-null   object
 1   Home Team  1600 non-null   object
 2   Away Team  1600 non-null   object
 3   Date       1600 non-null   object
 4   Report     1600 non-null   object
dtypes: object(5)
memory usage: 62.6+ KB


In [9]:
df = pd.DataFrame.from_dict(a)

dataset = Dataset.from_pandas(df)

dataset = dataset.train_test_split(test_size = 0.2)

dataset = dataset.flatten()


print(dataset['train'][0])


{'Title': "Aston Villa 2-3 Tottenham: Heung-Min Son's stoppage-time winner punishes Villa", 'Home Team': 'Aston Villa', 'Away Team': 'Tottenham Hotspur', 'Date': '2:00pm, Sunday 16th February 2020.', 'Report': 'Heung-Min Son scored a dramatic stoppage-time winner as Tottenham put significant pressure on Chelsea in the top-four race with a 3-2 win at Aston Villa.\nThe relegation-threatened hosts bossed the early stages and took a deserving lead when Toby Alderweireld scored an own goal (9).\nThe Belgian defender redeemed himself before the break when lashing home a difficult chance (27) before Spurs took advantage of a VAR call when being awarded a penalty when Bjorn Engels challenged Steven Bergwijn.\nPepe Reina saved Heung-Min Son\'s spot-kick but the South Korean was quickest to the rebound and made it 2-1 (45+2).\nIt looked a big ask for Villa to respond with Spurs playing cleverly on the counter-attack but Engels headed his team level (53) after a brilliant Jack Grealish corner.\nB

In [10]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [11]:
print(["".join(x) for x in dataset['train']['Report']])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
def preprocess_function(examples):

  return tokenizer(["".join(x) for x in examples['Report']], padding = True, truncation = True)

In [13]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_corpus = dataset.map(preprocess_function,
                          batched = True,
                          num_proc = 4,
                          remove_columns=dataset["train"].column_names
                          )

print(tokenized_corpus, tokenized_corpus['train']['input_ids'])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1280
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 320
    })
}) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
#block_size = 128
block_size = 256 #testing out a higher block size


def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [15]:
lm_dataset = tokenized_corpus.map(group_texts, batched=True, num_proc=4)

print(lm_dataset)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5120
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1280
    })
})


In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False, return_tensors = 'tf')

In [17]:
tf_train_set = lm_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    dummy_labels=True,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_test_set = lm_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    dummy_labels=True,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [18]:
optimizer = AdamWeightDecay(learning_rate = 2e-5, weight_decay_rate= 0.01)
model = TFAutoModelForCausalLM.from_pretrained('distilgpt2')

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [19]:
model.compile(optimizer = optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [21]:
model.fit(x= tf_train_set, validation_data=tf_test_set, epochs = 3) # 3 epochs seems to work - training loss gets close to validation loss

Epoch 1/3
320/320 [==============================] - 677s 2s/step - loss: 3.1951 - val_loss: 3.0464
Epoch 2/3
320/320 [==============================] - 676s 2s/step - loss: 3.0772 - val_loss: 2.9735
Epoch 3/3
320/320 [==============================] - 676s 2s/step - loss: 2.9996 - val_loss: 2.9262


In [30]:
input = 'Arsenal play Manchester United'
input_ids = tokenizer.encode(input, return_tensors='tf')

outputs = model.generate(input_ids = input_ids,
                         max_length = 500,
                         temperature = 1,
                         num_return_sequences = 2,
                         do_sample = True)

for i in range(2):
  print(f'Sample1: {i}: {tokenizer.decode(outputs[i], skip_special_tokens = True)} \n')


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Sample1: 0: Arsenal play Manchester United at St Mary's, but this will be an otherwise quiet start as United were denied their third clean sheet for the second time this season.
United had their fourth clean sheet, their highest since October and the third time they've faced another relegation run since January.
Pep Guardiola is a long way behind with the Frenchman not only drawing a fine goal against Tottenham but also taking the blame for a lack of quality in his side, but he also took the blame when he called home from an empty net to beat Southampton at St James' Park.
Guardiola has played out his frustrations of missing out, and this was something he managed to turn his head around at the Emirates later in the campaign.
As it stands, United have won only one away away game in the Premier League in 2017. City have lost only three of their last four home games this season and that record sets an old record for defeats by either side of the table.
Player Ratings
Manchester United
: C